<style> p { max-width: 500px; } </style>

# sed

<q>sed - stream editor for filtering and transforming text</q>
—manpages



<style> p { max-width: 500px; } </style>

Like most of these, `sed` is a fairly well known and supported tool in the UNIX world. However I've mostly just stuck to the basic substitution commands when using it.

But there's a whole range of potentially useful commands that it can execute.

GNU has HTML documentation here:
https://www.gnu.org/software/sed/manual/sed.html

In [4]:
# Standard 's' command we all know and love.
# Uses a s/word/replacement/ pattern.
echo Hello World | sed 's/World/Planet/'


# The 'g' flag can be used for multiple replacements.
echo How now brown cow? | sed 's/ow/**/'
echo How now brown cow? | sed 's/ow/**/g'

# GNU sed also supports backslashed commands in the pattern.
# \u for one letter uppercased, \U for all input until \E,
# \l and \L for lowercase.
echo hello world | sed 's/hello/\u\0/'
echo hello world | sed 's/hello/\U\0/'


Hello Planet
H** now brown cow?
H** n** br**n c**?
Hello world
HELLO world


<style> p { max-width: 500px; } </style>

I'm not sure if these extensions to change the capitalization are particuarly useful, but there are a few niche uses.

It's worth remembering that `/g` is not set by default. 


In [ ]:
cd ~/proj/shell

# You can also use a number to replace only the nth match.
echo cat cat cat cat cat cat cat | sed 's/cat/dog/4'

# \0 holds the pattern that was found.
# You can used it to add to the search term rather than removing it.
echo cat cat cat cat cat cat cat | sed 's/cat/bob\0/g'

# Text into the first set of '\(...\)` brackets in the pattern is \1
# You can have up to nine of these sets.
echo cat cat cat cat cat cat cat | sed 's/c\(at\)/b\1/g'

# GNU sed has a 'I' or 'i' flag that makes it case insensitive.
echo CAT CAT CAT CAT CAT CAT CAT | sed 's/cat/rabbit/g'
echo cat cat cat cat cat cat cat | sed 's/cat/rabbit/gi'

# The 'e' flag lets you pipe the match though a shell command.
echo cat | sed 's/cat/ls data\/\0*/e'

# A number before the command specifies a line. 
# x,y means to use lines x to line y (starts from one).
# the '-n' option prevents printing by default.
# The 'p' flag prints the results.
cat data/tree.txt | sed -n '1,4s/^[^ ]*/\U\0/p'


cat cat cat dog cat cat cat
bobcat bobcat bobcat bobcat bobcat bobcat bobcat
bat bat bat bat bat bat bat
CAT CAT CAT CAT CAT CAT CAT
rabbit rabbit rabbit rabbit rabbit rabbit rabbit
data/cat.txt
I think that I shall never see
A poem lovely as a tree.
EXCEPT for this one's sweet refrains;
LET'S print them on a tree's remains.


<style> p { max-width: 500px; } </style>

There's a lot of situations where you might want to only return matches, so I think the `sed -n 's/pattern/replace/p'` is quite handy.

Case sensitivity was another thing I forgot was the default, since I don't use it in Vim where I'm usually entering `s/` patterns.


In [ ]:
# sed supports line wrapping! Who knew?

# Print the first line wrapped to 70 characters.
sed -n --line-length 70 '1l' < 'data/lorem.txt'

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam posuere \
congue fringilla. Quisque vehicula tristique efficitur. Aliquam pelle\
ntesque lacus blandit, auctor massa quis, luctus justo. Suspendisse p\
otenti. Nullam pellentesque, justo id bibendum accumsan, odio nibh rh\
oncus augue, eu dictum leo ante eget quam. Donec ultricies, tellus id\
 sollicitudin placerat, odio ante dictum eros, id accumsan tortor ero\
s at eros. Nulla rutrum vehicula posuere. Sed finibus risus tortor, e\
t egestas eros consequat a. Proin at mi a est fermentum commodo sit a\
congue fringilla. Quisque vehicula tristique efficitur. Aliquam pelle\
ntesque lacus blandit, auctor massa quis, luctus justo. Suspendisse p\
otenti. Nullam pellentesque, justo id bibendum accumsan, odio nibh rh\
oncus augue, eu dictum leo ante eget quam. Donec ultricies, tellus id\
 sollicitudin placerat, odio ante dictum eros, id accumsan tortor ero\
s at eros. Nulla rutrum vehicula posuere. Sed finibus risus tortor, e\
t eges

<style> p { max-width: 500px; } </style>

I added this to the '00_get_first_word' sed file:
```sed
# Hashes are used for comments.

# Slash to find lines.
# This command just removes empty lines.
/^$/d
#  You need to use a "\(...\)" group for the pattern with ".*" after it.
# Otherwise you'll match the whole line.
s/\(^[^ ]*\).*/\U\1/p
```

There wasn't any syntax highlighting. 

And there's no extension in the store.

So I made my own, using an atom cson file I found here:
https://github.com/Alhadis/language-sed

I downloaded the 'grammers/sed.cson' file.

I ran:
```sh
sudo npm install -g cson
sudo npm install -g @vscode/vsce
mkdir sedsyntax
cd sedsyntax
mv ~/Downloads/sed.cson .
mkdir syntaxes
cson2json sed.cson > syntaxes/sed.json
```

Next I added this 'package.json' file from a modified example.

To build and install I ran:
```sh
vsce package
code --install-extension sedsyntax/sed-syntax-highlighter-0.0.1.vsix
```

After reselecting the format of the existing file... and what do you know it works!

<small>I was really expecting it to fail for some annoying unfixable reason.</small>

Anyway it looks like my sed file works:


In [37]:
cd ~/proj/shell
# sed can also run its own script files. 
# This seems typical of many of these Linux utilities.
# They have their own miniture programming language.

sed -nf src/00_get_first_word.sed < data/lorem.txt



LOREM
CURABITUR
PROIN
CURABITUR
PROIN
PRAESENT
MAURIS


In [10]:
cd ~/proj/shell
# 'a\' to append text.
echo There is a horse in the field. | sed 'a\Enter a command.'

# 'i\' to insert.
# These commands need a newline after then.
echo There is a horse in the field. | 
    sed 'i\---Status---
    a\Enter a command.
    s/horse/robo\0/'

echo

echo There is a horse in the field. | sed -nf src/01_robocom.sed



There is a horse in the field.
Enter a command.
Enter a command.
---Status---
There is a robohorse in the field.
Enter a command.

---Status---
There is a robohorse in the field.
Enter a command:


In [13]:
cd ~/proj/shell

# 'e' to execute a command from the current line.
# 'e command' to run a specific command.
# 'c\' to change the line.
echo ls | sed 's/ls/lsd/; e'

echo asdf | sed 'e date'

__pycache__
data
dist
get-pip.py
lib
LICENCE
notebook
data
dist
get-pip.py
lib
LICENCE
notebook
pyproject.toml
README.md
requirements.txt
sed-syntax-highlighter-0.0.1.vsix
sedsyntax
src
tests
Mon 27 Jan 2025 17:57:36 ACDT
asdf


<style> p { max-width: 500px; } </style>

Huh, interesting. 

So I can actually execute arbitrary commands with sed .

This seems useful (and dangerous).

Perhaps it might be good for a log formatter?

I created the '02_logger.sed' file.



In [15]:
cd ~/proj/shell

echo Text | sed -nf src/02_logger.sed
# Nope sed always appends a newline.


Mon 27 Jan 2025 18:12:33 ACDT
Text
Text
